In [2]:
import os
import math
import numpy as np
import pandas as pd

# === Parameters ===
pdb_path = "(path/to/structure.pdb"  # Path to your PDB file
chainA_id = "A"                  # First chain ID
chainB_id = "B"                  # Second chain ID
cutoff = 5.0                     # Å distance cutoff

out_csv = "interface_contacts.csv"
out_listA = "interface_chainA.txt"
out_listB = "interface_chainB.txt"
out_pdb_marked = "interface_marked.pdb"

# === Helpers ===
def parse_pdb_atoms(pdb_file):
    """Minimal PDB parser for ATOM/HETATM records."""
    atoms = []
    lines = []
    with open(pdb_file, "r") as f:
        for line in f:
            lines.append(line)
            rec = line[0:6]
            if rec.strip() not in ("ATOM", "HETATM"):
                continue
            try:
                atom_name = line[12:16].strip()
                altloc = line[16].strip()
                resname = line[17:20].strip()
                chain = line[21].strip()
                resseq = int(line[22:26])
                icode = line[26].strip()
                x = float(line[30:38])
                y = float(line[38:46])
                z = float(line[46:54])
                try:
                    bfac = float(line[60:66])
                except ValueError:
                    bfac = 0.0
                element = line[76:78].strip() if len(line) >= 78 else ""
                if not element:
                    element = atom_name[0].upper()
            except Exception:
                continue
            atoms.append({
                "atom_name": atom_name,
                "altloc": altloc,
                "resname": resname,
                "chain": chain,
                "resseq": resseq,
                "icode": icode,
                "x": x, "y": y, "z": z,
                "bfactor": bfac,
                "element": element,
                "line_index": len(lines)-1
            })
    return atoms, lines

def is_heavy(atom):
    e = atom["element"].upper()
    return e not in ("H", "D", "T")

def residue_label(atom):
    ch, name, idx, icode = atom["chain"], atom["resname"], atom["resseq"], atom["icode"]
    return f"{name} {ch}:{idx}{(icode if icode else '')}"

def chunked_contacts(coordsA, coordsB, cutoff):
    cutoff2 = cutoff * cutoff
    neighbors = [None] * coordsA.shape[0]
    chunk = 2048
    for start in range(0, coordsA.shape[0], chunk):
        end = min(start + chunk, coordsA.shape[0])
        a = coordsA[start:end]
        a2 = np.sum(a*a, axis=1)[:, None]
        b2 = np.sum(coordsB*coordsB, axis=1)[None, :]
        ab = a @ coordsB.T
        d2 = np.maximum(a2 + b2 - 2*ab, 0.0)
        within = d2 <= cutoff2
        for i in range(a.shape[0]):
            neighbors[start+i] = np.where(within[i])[0]
    return neighbors

def write_bfactor_marked_pdb(lines, atoms, interface_labels, out_path):
    iface_set = set(interface_labels)
    new_lines = lines[:]
    for atom in atoms:
        if residue_label(atom) not in iface_set:
            continue
        idx = atom["line_index"]
        line = lines[idx]
        if not (line.startswith("ATOM") or line.startswith("HETATM")):
            continue
        line = line.rstrip("\n")
        if len(line) < 66:
            line = line.ljust(66)
        bfac_str = f"{100.0:6.2f}"
        line = line[:60] + bfac_str + line[66:]
        new_lines[idx] = line + "\n"
    with open(out_path, "w") as f:
        f.writelines(new_lines)

# === Main ===
atoms_all, pdb_lines = parse_pdb_atoms(pdb_path)
atomsA = [a for a in atoms_all if a["chain"] == chainA_id and is_heavy(a) and (a["altloc"] in ("", " ", "A"))]
atomsB = [a for a in atoms_all if a["chain"] == chainB_id and is_heavy(a) and (a["altloc"] in ("", " ", "A"))]

coordsA = np.array([[a["x"], a["y"], a["z"]] for a in atomsA], dtype=float)
coordsB = np.array([[b["x"], b["y"], b["z"]] for b in atomsB], dtype=float)

neighbor_idx_list = chunked_contacts(coordsA, coordsB, cutoff=cutoff)

pair_min = {}
iface_resA = set()
iface_resB = set()

for iA, neigh in enumerate(neighbor_idx_list):
    if neigh.size == 0:
        continue
    atomA = atomsA[iA]
    resA_label = residue_label(atomA)
    for jB in neigh:
        atomB = atomsB[int(jB)]
        dx, dy, dz = coordsA[iA] - coordsB[int(jB)]
        d = math.sqrt(dx*dx + dy*dy + dz*dz)
        resB_label = residue_label(atomB)
        key = (resA_label, resB_label)
        if (key not in pair_min) or (d < pair_min[key]):
            pair_min[key] = d
        iface_resA.add(resA_label)
        iface_resB.add(resB_label)

# Save outputs
df_contacts = pd.DataFrame(
    [{"Residue_ChainA": k[0], "Residue_ChainB": k[1], "MinDistance_Angstrom": v}
     for k, v in pair_min.items()]
).sort_values(by=["Residue_ChainA", "Residue_ChainB", "MinDistance_Angstrom"])

df_contacts.to_csv(out_csv, index=False)

with open(out_listA, "w") as f:
    for lbl in sorted(iface_resA):
        f.write(lbl + "\n")
with open(out_listB, "w") as f:
    for lbl in sorted(iface_resB):
        f.write(lbl + "\n")

write_bfactor_marked_pdb(pdb_lines, atoms_all, iface_resA.union(iface_resB), out_pdb_marked)

# === Summary ===
print(f"Interface residues in chain {chainA_id}: {len(iface_resA)}")
print(f"Interface residues in chain {chainB_id}: {len(iface_resB)}")
print(f"Residue–residue contact pairs: {len(df_contacts)}")
print(f"CSV saved to: {out_csv}")
print(f"List A saved to: {out_listA}")
print(f"List B saved to: {out_listB}")
print(f"B-factor marked PDB saved to: {out_pdb_marked}")

# Preview
df_contacts.head()


Interface residues in chain A: 22
Interface residues in chain B: 33
Residue–residue contact pairs: 70
CSV saved to: interface_contacts.csv
List A saved to: interface_chainA.txt
List B saved to: interface_chainB.txt
B-factor marked PDB saved to: interface_marked.pdb


,Residue_ChainA,Residue_ChainB,MinDistance_Angstrom
46,ALA A:47,GLU B:113,4.293022
52,ALA A:53,GLN B:119,4.661435
51,ALA A:53,PHE B:117,4.241195
56,ALA A:55,GLN B:119,4.451249
59,ALA A:55,ILE B:186,3.892751
